### Display, select and move images
Visually move your images from one directory to another.

#### Simple code

In [1]:
import os
import cv2
import shutil

In [2]:
# Constants for text annotation
FONT = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.7
FONT_COLOR_YELLOW = (0, 255, 255)  # Yellow color for image index
FONT_COLOR_WHITE = (255, 255, 255)  # White color for instructions
FONT_BG_COLOR = (0, 0, 255)
FONT_THICKNESS = 2

In [3]:
def show_and_store_images(directory, destination_path):
    """
    Displays images one by one using OpenCV imshow and provides interactive options.
    """
    image_files = [file for file in sorted(os.listdir(directory)) if os.path.isfile(os.path.join(directory, file))]

    file_count = len(image_files)
    idx = 0
    selected_indices = []

    while True:
        if idx > file_count - 1:
            idx = 0
        elif idx < 0:
            idx = file_count - 1

        img_path = os.path.join(directory, image_files[idx])
        img = cv2.imread(img_path)

        if img is not None:  # Check if image is successfully read
            image = img.copy()

            # Display current image index on the image
            index_text = f'Image {idx+1}/{file_count}'
            text_size = cv2.getTextSize(index_text, FONT, FONT_SCALE, FONT_THICKNESS)[0]
            text_x = 20
            text_y = text_size[1] + 20

            # Display image index on the top left corner of the image
            image = cv2.putText(image, index_text, (text_x, text_y), FONT, FONT_SCALE, FONT_COLOR_YELLOW, FONT_THICKNESS, cv2.LINE_AA)

            # Display instructions on the image
            instructions = [
                "Left arrow: Previous image",
                "Right arrow: Next image",
                "s: Select image",
                "c: Confirm selection and move",
                "Esc: Quit"
            ]
            max_instr_len = max(map(len, instructions)) + 15
            max_instr_height = len(instructions) * 25 + 10

            cv2.rectangle(image, (img.shape[1] - max_instr_len * 9, 0), (img.shape[1], max_instr_height), FONT_BG_COLOR, -1)
            for i, instr in enumerate(instructions):
                cv2.putText(image, instr, (img.shape[1] - max_instr_len * 8, (i + 1) * 25), FONT, FONT_SCALE, FONT_COLOR_WHITE, FONT_THICKNESS, cv2.LINE_AA)

            cv2.namedWindow('Image Viewer', cv2.WINDOW_KEEPRATIO)
            cv2.imshow('Image Viewer', image)

        key = cv2.waitKey()
        if key == 27:  # Escape key
            cv2.destroyAllWindows()
            break

        if key == 83:  # Right arrow key
            idx += 1

        if key == 81:  # Left arrow key
            idx -= 1

        if key == ord('s'):
            print('Selected image: {}'.format(idx))
            selected_indices.append(idx)

        if key == ord('r'):
            print("Resetting selection!")
            selected_indices.clear()
            print(selected_indices)

        if key == ord('c'):  # Confirm and move selected images
            if selected_indices:
                print('Confirm selection and move the following images:')
                for i in selected_indices:
                    print(f"{image_files[i]}")
                confirm_key = cv2.waitKey(0)
                if confirm_key == ord('c'):
                    if not os.path.exists(destination_path):
                        os.makedirs(destination_path)
                    for i in selected_indices:
                        img_path = os.path.join(directory, image_files[i])
                        try:
                            shutil.move(img_path, destination_path)
                        except Exception as e:
                            print(e)
                    selected_indices = []
                    print('Selected images moved.')
                else:
                    print('Selection canceled.')
                cv2.destroyAllWindows()  # Close OpenCV window after moving images
                break  # Terminate the program after moving images


In [4]:
if __name__ == "__main__":
    base_directory_path = '/home/acer/workspace/videos/src'
    dest_path = '/home/acer/workspace/videos/dest'  # Define the destination path here
    show_and_store_images(base_directory_path, dest_path)


Selected image: 0
Selected image: 0
Selected image: 0
Selected image: 0
Selected image: 0
Selected image: 0
Selected image: 2
Selected image: 3
Selected image: 4
Confirm selection and move the following images:
image_0150.jpg
image_0150.jpg
image_0150.jpg
image_0150.jpg
image_0150.jpg
image_0150.jpg
image_0158.jpg
image_0162.jpg
image_0163.jpg
Destination path '/home/acer/workspace/videos/dest/image_0150.jpg' already exists
Destination path '/home/acer/workspace/videos/dest/image_0150.jpg' already exists
Destination path '/home/acer/workspace/videos/dest/image_0150.jpg' already exists
Destination path '/home/acer/workspace/videos/dest/image_0150.jpg' already exists
Destination path '/home/acer/workspace/videos/dest/image_0150.jpg' already exists
Selected images moved.


#### Modular code with Logging and Error Handling

In [6]:
import os
import cv2
import shutil
import logging

# Constants
FONT = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.7
FONT_THICKNESS = 2
FONT_COLOR_YELLOW = (0, 255, 255)
FONT_COLOR_WHITE = (255, 255, 255)
FONT_BG_COLOR = (0, 0, 255)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def display_image(image, idx, file_count, error_message=None):
    """
    Display the image with index information and instructions.

    Args:
    - image: The image to display.
    - idx: Index of the current image.
    - file_count: Total number of images.
    - error_message: Optional error message to display.
    """
    index_text = f'Image {idx + 1}/{file_count}'
    text_size = cv2.getTextSize(index_text, FONT, FONT_SCALE, FONT_THICKNESS)[0]
    text_x = 20
    text_y = text_size[1] + 20

    image = cv2.putText(image, index_text, (text_x, text_y), FONT, FONT_SCALE, FONT_COLOR_YELLOW, FONT_THICKNESS, cv2.LINE_AA)

    instructions = [
        "Left arrow: Previous image",
        "Right arrow: Next image",
        "s: Select image",
        "c: Confirm selection and move",
        "r: Reset selection",
        "Esc: Quit"
    ]
    max_instr_len = max(map(len, instructions)) + 15
    max_instr_height = len(instructions) * 25 + 10

    cv2.rectangle(image, (image.shape[1] - max_instr_len * 9, 0), (image.shape[1], max_instr_height), FONT_BG_COLOR, -1)
    for i, instr in enumerate(instructions):
        cv2.putText(image, instr, (image.shape[1] - max_instr_len * 8, (i + 1) * 25), FONT, FONT_SCALE, FONT_COLOR_WHITE, FONT_THICKNESS, cv2.LINE_AA)

    if error_message:
        cv2.putText(image, error_message, (20, image.shape[0] - 20), FONT, FONT_SCALE, FONT_COLOR_YELLOW, FONT_THICKNESS, cv2.LINE_AA)

    cv2.imshow('Image Viewer', image)

def move_selected_images(selected_indices, image_files, directory, destination_path):
    """
    Confirm and move selected images.

    Args:
    - selected_indices: List of indices to move.
    - image_files: List of image filenames.
    - directory: Path to the source directory.
    - destination_path: Path to the destination directory.

    Returns:
    - True if images were moved successfully, False otherwise.
    """
    if not selected_indices:
        logger.info("No images selected for moving.")
        return False

    logger.info('Confirm selection and move the following images:')
    for i in selected_indices:
        logger.info(f"{image_files[i]}")

    confirm_key = cv2.waitKey(0)
    if confirm_key == ord('c'):
        if not os.path.exists(destination_path):
            os.makedirs(destination_path)

        for i in selected_indices:
            img_path = os.path.join(directory, image_files[i])
            try:
                shutil.move(img_path, destination_path)
            except Exception as e:
                logger.error(f"Error moving {img_path}: {e}")
                continue

        logger.info('Selected images moved.')
        return True
    else:
        logger.info('Move operation canceled.')
        return False

def update_image_files(image_files, moved_indices):
    """
    Remove moved indices from image_files.

    Args:
    - image_files: List of image filenames.
    - moved_indices: List of indices that were moved.

    Returns:
    - Updated list of image filenames.
    """
    return [file for i, file in enumerate(image_files) if i not in moved_indices and i < len(image_files)]

def show_image(directory, image_files, idx, error_message=None):
    """
    Show the image with the given index.

    Args:
    - directory: Path to the source directory.
    - image_files: List of image filenames.
    - idx: Index of the current image.
    - error_message: Optional error message.

    Returns:
    - None
    """
    if 0 <= idx < len(image_files):
        img_path = os.path.join(directory, image_files[idx])
        try:
            img = cv2.imread(img_path)

            if img is not None:
                image = img.copy()
                display_image(image, idx, len(image_files), error_message)
        except Exception as e:
            logger.error(f"Error reading image {img_path}: {e}")
    else:
        display_image(None, idx, len(image_files), error_message="Invalid index")

def show_and_store_images(directory, destination_path):
    """
    Displays images one by one using OpenCV imshow and provides interactive options.

    Args:
    - directory: Path to the source directory.
    - destination_path: Path to the destination directory.

    Returns:
    - None
    """
    image_files = [file for file in sorted(os.listdir(directory)) if os.path.isfile(os.path.join(directory, file))]

    file_count = len(image_files)
    idx = 0
    selected_indices = []
    images_moved = False

    cv2.namedWindow('Image Viewer', cv2.WINDOW_KEEPRATIO)

    while True:
        if idx > file_count - 1:
            idx = 0
        elif idx < 0:
            idx = file_count - 1

        show_image(directory, image_files, idx)

        key = cv2.waitKey()

        if key == 27:  # Escape key
            confirm_exit = input("Are you sure you want to quit? (y/n): ")
            if confirm_exit.lower() == 'y':
                break
        elif key == 83:  # Right arrow key
            idx += 1
        elif key == 81:  # Left arrow key
            idx -= 1
        elif key == ord('s'):
            logger.info('Selected image: {}'.format(idx))
            selected_indices.append(idx)
        elif key == ord('r'):
            logger.info("Resetting selection!")
            selected_indices.clear()
            logger.info(selected_indices)
        elif key == ord('c'):
            if move_selected_images(selected_indices, image_files, directory, destination_path):
                images_moved = True
                break

    if images_moved:
        image_files = update_image_files(image_files, selected_indices)

    cv2.destroyAllWindows()

if __name__ == "__main__":
    base_directory_path = '/home/acer/workspace/videos/src'
    dest_path = '/home/acer/workspace/videos/dest'
    show_and_store_images(base_directory_path, dest_path)


INFO:__main__:Selected image: 1
INFO:__main__:Selected image: 2
INFO:__main__:Selected image: 3
INFO:__main__:Selected image: 4
INFO:__main__:Selected image: 5
INFO:__main__:Selected image: 6
INFO:__main__:Confirm selection and move the following images:
INFO:__main__:image_0170.jpg
INFO:__main__:image_0171.jpg
INFO:__main__:image_0172.jpg
INFO:__main__:image_0173.jpg
INFO:__main__:image_0174.jpg
INFO:__main__:image_0175.jpg
INFO:__main__:Selected images moved.
